In [1]:
import pandas as pd
import numpy as np
from numpy import NaN

In [2]:
ac = pd.read_csv('results/1С-аналитик_vac.csv', sep=';', encoding='utf-8')
ds = pd.read_csv('results/Data Scientist_vac.csv', sep=';', encoding='utf-8')
do = pd.read_csv('results/DataOps-инженер_vac.csv', sep=';', encoding='utf-8')
ab = pd.read_csv('results/Аналитик BI_vac.csv', sep=';', encoding='utf-8')
ad = pd.read_csv('results/Аналитик данных_vac.csv', sep=';', encoding='utf-8')
dj = pd.read_csv('results/Дата-журналист_vac.csv', sep=';', encoding='utf-8')
de = pd.read_csv('results/Дата-инженер_vac.csv', sep=';', encoding='utf-8')
ma = pd.read_csv('results/Маркетинговый аналитик_vac.csv', sep=';', encoding='utf-8')
pa = pd.read_csv('results/Продуктовый аналитик_vac.csv', sep=';', encoding='utf-8')
sa = pd.read_csv('results/Системный аналитик_vac.csv', sep=';', encoding='utf-8')
fa = pd.read_csv('results/Финансовый аналитик_vac.csv', sep=';', encoding='utf-8')


In [3]:
df = pd.concat([ac, ds, do, ab, ad, dj, de, ma, pa, sa, fa])
df

,id,spec,job_title,city,employer,schedule,experience,published,salary_from,salary_to,currency,skills
0,78499122,1С-аналитик,1С Аналитик,Иннополис,ХАЙЕР ИНДАСТРИ РУС,NaN,NaN,2023-03-24,NaN,NaN,NaN,NaN
1,78499123,1С-аналитик,1С Аналитик,Казань,ХАЙЕР ИНДАСТРИ РУС,NaN,NaN,2023-03-24,NaN,NaN,NaN,NaN
2,78499126,1С-аналитик,1С Аналитик,Ижевск,ХАЙЕР ИНДАСТРИ РУС,NaN,NaN,2023-03-24,NaN,NaN,NaN,NaN
3,78056414,1С-аналитик,Программист 1C / Аналитик 1С,Нижний Новгород,Компания по удаленной поддержке организаций,NaN,NaN,2023-03-14,130000.0,210000.0,RUR,1с: предприятие 8
4,68818149,1С-аналитик,Программист 1С / Аналитик 1С,Томск,ТрансСервис,NaN,NaN,2023-04-06,90000.0,120000.0,RUR,"1с программирование, 1с: документооборот, 1с: ..."
...,...,...,...,...,...,...,...,...,...,...,...,...
656,75967116,Финансовый аналитик,Финансовый менеджер/Аналитик,Ростов-на-Дону,Роллер Раббер Регион,NaN,NaN,2023-04-05,80000.0,NaN,RUR,"финансовая отчетность, управленческая отчетнос..."
657,78727594,Финансовый аналитик,Финансовый менеджер-аналитик,Ростов-на-Дону,Даная,NaN,NaN,2023-03-30,55000.0,55000.0,RUR,NaN
658,73449732,Финансовый аналитик,Финансовый аналитик по инвестиционным проектам,Ростов-на-Дону,Агрохолдинг СТЕПЬ,NaN,NaN,2023-03-31,NaN,NaN,NaN,NaN
659,78365242,Финансовый аналитик,Финансовый аналитик / экономист,Ростов-на-Дону,Грандфаянс,NaN,NaN,2023-03-21,NaN,NaN,NaN,"финансовый анализ, финансовый контроль, эконом..."


In [7]:
df.experience.unique()

array([nan])

In [5]:
pd.pivot_table(
    df,
    index='skills',
    columns='spec',
    values='id',
    aggfunc='count'
)

spec,1С-аналитик,Data Scientist,DataOps-инженер,Аналитик BI,Аналитик данных,Дата-инженер,Маркетинговый аналитик,Продуктовый аналитик,Системный аналитик,Финансовый аналитик
skills,,,,,,,,,,
"1c erp, erp-системы на базе 1с",1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"1c erp, анализ данных, анализ затрат, аналитическое мышление",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
"1c erp, финансовая отчетность, моделирование",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
"1c: бухгалтерия, 1с: предприятие 8, 1с: бухгалтерия, 1c: зарплата и кадры, 1c: финансы, ms excel, банк-клиент, эквайринг, отправка платежей",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
"1c: бухгалтерия, 1с: управление персоналом, 1с: документооборот, пользователь пк",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...
"экономический анализ, английский язык, финансовый анализ, статистический анализ, работа с большим объемом информации",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
"экономический анализ, управленческая отчетность, анализ финансовых показателей, аналитическое мышление, умение работать в условиях многозадачности, подготовка управленческой отчетности, готовность к командировкам, финансовая отчетность, опытный пользователь пк",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
"экономический анализ, финансовый анализ, финансовое планирование, финансовый менеджмент, аналитическое мышление, бюджетирование",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [5]:
df.to_csv('hh_vacancies.csv')